# Getting Started with Agentune Simulate - Quick Start

This notebook provides a streamlined introduction to the Agentune Simulate library. You'll learn to:

- Load conversation data and extract outcomes
- Set up RAG-based simulation with in-memory vector store
- Run simulations and analyze results

**Note**: This is a simplified version. For persistent storage and production features, see `persistent_storage_example.ipynb`.

## Setup and Imports

In [ ]:
import os
import getpass
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from agentune.simulate.models import Outcomes
from agentune.simulate.rag import conversations_to_langchain_documents
from agentune.simulate.simulation.session_builder import SimulationSessionBuilder
from utils import setup_logging_and_asyncio, load_data_with_outcomes

## API Key Configuration

In [ ]:
# Set up OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✓ API key configured")

## Environment Setup

In [ ]:
# Configure logging and asyncio for Jupyter
setup_logging_and_asyncio()

## Load Data and Extract Outcomes

In [ ]:
# Load conversations and extract outcomes in one step
conversations, outcomes_tuple = load_data_with_outcomes("data/sample_conversations.csv")
outcomes = Outcomes(outcomes=outcomes_tuple)

## Create Models and Vector Store

In [ ]:
# Setup models
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create in-memory vector store
documents = conversations_to_langchain_documents(conversations)
vector_store = InMemoryVectorStore.from_documents(documents, embeddings)

print(f"✓ Created vector store with {len(documents)} documents")

## Run Simulation

In [ ]:
# Build and run simulation session
session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=vector_store,
    max_messages=10
).build()

# Run simulation with first 5 conversations as starting points
base_conversations = conversations[:5]
result = await session.run_simulation(real_conversations=base_conversations)

print("✓ Simulation completed!")

## Analyze Results

In [ ]:
print(result.generate_summary())

## Next Steps

Now that you've completed a basic simulation:

  1. **Use your own data**: Load your own conversations as a list of `Conversation`
  objects and use them to set up the simulation.
  2. **Production features**: See `persistent_storage_example.ipynb` for persistent
  storage with Chroma and advanced configurations suitable for larger datasets and
  production use.
  3. **Explore advanced features**: Check out the full documentation for more options
  like caching of LLM responses, LLM failures handling, and more.

  ### Resources:
  - [Full documentation](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/README.md)
  - [Complete examples](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/examples)
  - [Persistent storage example](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/examples/persistent_storage_example.ipynb)
  - [Streamlit web interface](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/streamlit/README.md)